In [106]:
''' 
Development Juptyer Notebooks for The Benchmark Calculation Overtime . 
''' 

from abc import abstractmethod, ABC 
import pandas as pd 
from typing import Iterable, List, Optional
from datetime import datetime
import numpy as np

In [ ]:
'''  
pulling cleaned data into the notebook 
'''
df = pd.read_csv("bloomberg_data_cleaned.csv")

#set date to datetime object 
df["date"] = pd.to_datetime(df["date"])
df.set_index(["date","ticker"],inplace= True)
pd.set_option('display.float_format', '{:,.4f}'.format)
df.head()

market_cap          volume  price_to_book  \
date       ticker                                                 
2022-01-03 A         47,257.0847    554,098.0000         8.7692   
           AA        11,293.5460  1,033,749.0000         2.3785   
           AAL       12,140.8973  8,786,970.0000            NaN   
           AAON       4,106.6876     83,790.0000         8.8262   
           AAPL   2,986,128.3180 20,242,137.0000        41.3474   

                   price_to_sales  pe_ratio  current_cap_share_class  \
date       ticker                                                      
2022-01-03 A               7.5274   42.0279              47,257.0847   
           AA              0.9249    8.9225              11,293.5460   
           AAL             0.4051       NaN              12,140.8973   
           AAON            7.6801   63.5297               4,106.6876   
           AAPL            7.9614   30.1565           2,986,128.3180   

                   close_price  
date       ticker               
2022-01-03 A          156.4800  
           AA          60.3600  
           AAL         18.7500  
           AAON        52.2200  
           AAPL       182.0100

In [ ]:



class Benchmark:
    """_summary_
    """
    def __init__(self, data: pd.DataFrame, benchmark_start_val: float): 
        self.data: pd.DataFrame = data
        self.cur_constituents: List[str] #this is all of current member of the index. 
        self.dates: List[datetime]= self.data.index.get_level_values("date").unique().to_list()
        self.cur_date: datetime  = self.dates[0]
        self._i: int = 0 
        self.price_history = pd.Series(data = 0.0, index = self.dates)
        self.benchmark_divisor: float = 1
        self.quarterly_recon_dates: List[datetime] = []
        self._get_quarterly_reconst_dates() 
        self.random_constitution()
        self._calc_benchmark_divisor(benchmark_start_val) 
    
    #=========================================================
    #          Benchmark Constitution Methods
    #========================================================


    def next_benchmark_price(self) -> float: 
        if self.is_reconstitution_date():
            # reconsitute the benchmark. 
            None 
        next_benchmark = self._calc_numerator() / self.benchmark_divisor
        self.price_history.loc[self.cur_date] = next_benchmark #type: ignore 
        self.next_date() 
        return next_benchmark
    
    def _calc_numerator(self)-> float: 
        numerator: float = 0
        for ticker in self.cur_constituents: 
            key = (self.cur_date,ticker)
            cap_val  = self.data.loc[key]["market_cap"] # type: ignore 
            if np.isnan(cap_val): 
                cap_val = 0.00 
            numerator +=  cap_val 
        return numerator


    def _calc_benchmark_divisor(self, start_val: float) -> None: 
        # numerator / start_vale = divisor. 
        self.benchmark_divisor = self.next_benchmark_price() / start_val
    
    #=========================================================
    #          Benchmark Constitution Methods
    #========================================================

    def random_constitution(self):
        tickers = self.data.index.get_level_values("ticker").unique()
        mask = [] 
        rnd_selection =  np.random.randint(0,tickers.__len__(),282)
        for i, ticker in enumerate(range(tickers.__len__())): 
            if i in rnd_selection: 
                mask += [True]
            else: 
                mask += [False]
        self.cur_constituents = tickers[mask].to_list()

    #========================================================
    #           Date Handling Methods: 
    #========================================================
    
    def _get_quarterly_reconst_dates(self) -> None: 
        """
        Last trading of the months January, April, July, October
        """
        s_dates: pd.Series = pd.Series(self.dates)
        for year in range(self.dates[0].year, self.dates[-1].year + 1):
            for month in [1,4,7,10]:
                year_month_mask = (s_dates.dt.year == year) &  (s_dates.dt.month == month) #type: ignore 
                self.quarterly_recon_dates += [s_dates[year_month_mask].iloc[-1]]


    def is_reconstitution_date(self) -> bool:  
        out = False
        annual_recon_dates = [
                             datetime(2021,5,25),
                             datetime(2022,5,24),
                             datetime(2023,5,23),
                             datetime(2024,5,28),
                             datetime(2025,5,27)]
        if self.cur_date in annual_recon_dates: 
            return True
        if self.cur_date in self.quarterly_recon_dates: 
            return True 
        return False 
              

    def next_date(self) -> Optional[datetime]:
        if self._i < self.dates.__len__(): 
            self.cur_date = self.dates[self._i]
            self._i += 1 
            return self.cur_date
        else:
            return None 
        

In [ ]:
bench = Benchmark(df,100)
iter = 0
while bench.cur_date is not None: 
    iter += 1 
    if iter > 100000: break
    #print(bench.cur_date)
    bench.next_benchmark_price()

bench.price_history


In [98]:
bench = Benchmark(df)
bench._get_quarterly_reconst_dates_2()
bench.quarterly_recon_dates

d = datetime.strptime('2025-10-31 00:00:00', '%Y-%m-%d %H:%M:%S')
d in bench.quarterly_recon_dates

True

In [125]:
bench = Benchmark(df,1000000)

In [ ]:
tickers = df.index.get_level_values("ticker").unique()
mask = [] 
rnd_selection =  np.random.randint(0,tickers.__len__(),282)
for i, ticker in enumerate(range(tickers.__len__())): 
    if i in rnd_selection: 
        mask += [True]
    else: 
        mask += [False]
mask
tickers[mask]

Index(['AAON', 'AAPL', 'ACI', 'ADT', 'AES', 'AFL', 'AGCO', 'AGO', 'AIZ', 'AJG',
       ...
       'WLK', 'WMB', 'WMS', 'WMT', 'WU', 'WYNN', 'XP', 'XRAY', 'Z', 'ZS'],
      dtype='object', name='ticker', length=250)

In [119]:
rnd_selection.__len__()
mask.__len__()

1010

In [147]:
bench = Benchmark(df,1000000)
len(bench.cur_constituents)
bench.benchmark_divisor
bench._calc_numerator()
bench.next_date()

Timestamp('2022-01-04 00:00:00')

In [148]:
bench.next_benchmark_price()
bench.next_benchmark_price()
bench.next_benchmark_price()
bench.next_benchmark_price()

np.float64(979452.2028593788)

In [157]:
bench.price_history
price_history = pd.Series(
    data = 0.0,
    index = bench.dates 
)
d = datetime.strptime('2025-10-31 00:00:00', '%Y-%m-%d %H:%M:%S')
print(d)
price_history.loc[d] = 4 
price_history.loc[d]

2025-10-31 00:00:00


np.float64(4.0)

In [ ]:
'''
realistically the bechmark itself should be place into a dataframe of the form 
index = Dates | Benchmark_Price | Divisor | Market Cap 

''' 
'''
==========================================
        Data Availability Dataframe: 
==========================================
other thoughts on data availability. Overall what we have is 

'''


data_stats_df = pd.DataFrame(
    columns = ["has_market_cap_data","has_all_data"] 

)




In [ ]:
df.loc[d,:]